# dspy methodology 101

1. programming
   1. LMs (tasks)
   2. signatures (i/o) - compiling leads to better prompts than humans write
   3. modules
2. evaluation
3. optimization

## set a generator LM

In [1]:
import dspy
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if OPENAI_API_KEY:
    print("OPENAI API Key found!")
else:
    print("OPENAI API Key not found!")
if ANTHROPIC_API_KEY:
    print("ANTHROPIC API Key found!")
else:
    print("ANTHROPIC API Key not found!")

lm = dspy.LM('openai/gpt-4o-mini', temperature=0.9, max_tokens=3000, stop=None, cache=False, api_key=OPENAI_API_KEY)
dspy.configure(lm=lm)

OPENAI API Key found!
ANTHROPIC API Key found!


## testing with dspy modules

Modules help you describe AI behavior as code, not strings.

In [2]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

len(lm.history)
lm.history[0].keys()

5765.0


dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type'])

In [10]:
with dspy.context(lm=dspy.LM('openai/gpt-3.5-turbo')):
    math = dspy.ChainOfThought("question -> answer: float")
    result = math(question="Two dice are tossed. What is the probability that the sum equals two?")
    print("Result:", result)

print(f"\nHistory length: {len(lm.history)}")
lm.history[0].keys()

with dspy.context(lm=dspy.LM('anthropic/claude-3-7-sonnet-20250219', temperature=0.9, max_tokens=3000, stop=None, cache=False, api_key=ANTHROPIC_API_KEY)):
    math = dspy.ChainOfThought("question -> answer: float")
    result = math(question="Two dice are tossed. What is the probability that the sum equals two?")
    print('claude-3-opus Result:', result) # where result.answer would display the answer but no reasoning

print(f"\nHistory length: {len(lm.history)}")
lm.history[-1].keys()

Result: Prediction(
    reasoning='The sum of two dice can range from 2 to 12. To get a sum of 2, only one combination is possible: (1, 1). There are a total of 36 possible outcomes when two dice are tossed (6 outcomes for the first die and 6 outcomes for the second die). Therefore, the probability of getting a sum of 2 is 1/36.',
    answer=0.027777777777777776
)

History length: 4
claude-3-opus Result: Prediction(
    reasoning="When two dice are tossed, each die can show a face value from 1 to 6. To get a sum of 2, we need both dice to show 1.\n\nLet's denote the outcome of the first die as D1 and the second die as D2.\nWe need D1 = 1 and D2 = 1.\n\nTotal number of possible outcomes when tossing two dice = 6 × 6 = 36 outcomes.\nNumber of favorable outcomes (getting a sum of 2) = 1 outcome (when D1 = 1 and D2 = 1).\n\nTherefore, the probability of getting a sum of 2 is:\nProbability = Number of favorable outcomes / Total number of possible outcomes\nProbability = 1/36 ≈ 0.02777...",


dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type'])

In [15]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.85
)

In [16]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)

Prediction(
    reasoning='The context states that David Gregory inherited Kinnairdy Castle in 1664. Therefore, the name of the castle he inherited is Kinnairdy Castle.',
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'
)

In [7]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Inc. Announces iPhone 14
['Announcement', 'CEO Statement', 'New Features']
[{'name': 'Apple Inc.', 'type': 'Organization'}, {'name': 'iPhone 14', 'type': 'Product'}, {'name': 'Tim Cook', 'type': 'Person'}]


In [13]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="World Cup 2002")

print(f"# {article.title}\n")
for section in article.sections:
    print(f"{section}\n")

# Overview of the 2002 FIFA World Cup

## Introduction

### Background
The 2002 FIFA World Cup marked the 17th edition of the tournament and was notable for being the first World Cup jointly hosted by two nations, South Korea and Japan. This landmark event took place from May 31 to June 30, 2002, and it was a pivotal moment in the world of football, showcasing the sport's ability to unite nations and fans across the globe. Following the end of the 1998 World Cup in France, which saw the host nation emerge victorious, the 2002 tournament was an opportunity for teams to vie for the prestigious trophy, further enhancing the global football narrative.

### Host Nations
South Korea and Japan worked collaboratively to prepare for the World Cup, a historic undertaking that required extensive infrastructure development and cultural cooperation. This collaboration was not only a logistical challenge but also a representation of how football can foster camaraderie between countries. The tourname